## 1. Archive Exploration

Take some time to look through the dataset’s description in EDI and click around. Discuss the following questions with your team:

What is this data about?

Snowshoe hair physical data in Bonzana Creek Experimental Forest.

During what time frame were the observations in the dataset collected?

1999 - 2012

Does the dataset contain sensitive data?

No

Is there a publication associated with this dataset?

Flora, B.K. 2002. Comparison of snowshoe hare populations in Interior. M.S. Thesis. University of Alaska Fairbanks. Fairbanks, AK, USA.


In your notebook: use a markdown cell to add a brief description of the dataset, including a citation, date of access, and a link to the archive.

brief description: Capture/recapture studies of snowshoe hares at 5 locales in Tanana valley.  

citation: Kielland, K., F.S. Chapin, R.W. Ruess, and Bonanza Creek LTER. 2017. Snowshoe hare physical data in Bonanza Creek Experimental Forest: 1999-Present ver 22. Environmental Data Initiative. https://doi.org/10.6073/pasta/03dce4856d79b91557d8e6ce2cbcdc14 (Accessed 2025-10-17).

date of access: 10/16/2025

link: https://portal.edirepository.org/nis/mapbrowse?packageid=knb-lter-bnz.55.22

## 2. Adding an image 

![Snowshoe hare](https://upload.wikimedia.org/wikipedia/commons/thumb/8/8a/SNOWSHOE_HARE_%28Lepus_americanus%29_%285-28-2015%29_quoddy_head%2C_washington_co%2C_maine_-01_%2818988734889%29.jpg/1452px-SNOWSHOE_HARE_%28Lepus_americanus%29_%285-28-2015%29_quoddy_head%2C_washington_co%2C_maine_-01_%2818988734889%29.jpg?20170313021652)

## 3. Data loading and preliminary exploration

## 4. Detecting messy values 

## 5. Brainstorm

## 6. Clean Values 

## 7. Calculate mean weight 

## 8. Collect your code and explain results 